# <center>Phân tích ma trận trong Python</center>

## Mục lục
* [Thực hành](#c1)
    * [Trị riêng - Vector riêng](#c11)
    * [Chéo hóa](#c12)
    * [Chéo hóa trực giao](#c13)
* [Đồ án 2: Image Processing](#c2)
    * [Nội dung đồ án](#c21)
    * [Quy định nộp bài](#c22)
    * [Quy định chấm bài](#c23)

## Thực hành <a class="anchor" id="c1"></a>

Trong lab này, chúng ta sẽ tìm hiểu về phân tích ma trận sử dụng `NumPy`.

Cho ma trận:
$$A = \begin{bmatrix}
    3 & -2 & 0\\ 
    -2 & 3 & 0\\ 
    0 & 0 & 5
    \end{bmatrix}$$

In [ ]:
import numpy as np

In [ ]:
A = np.array([[3, -2, 0],
              [-2, 3, 0],
              [0, 0, 5]])

In [ ]:
def is_close(A, B, eps=10e-3):
    return np.all(np.abs(A - B) < eps)

Các phân tích được giới thiệu trong lab này là:
1. Tìm trị riêng và vector riêng
2. Chéo hóa
3. Chéo hóa trực giao

### Trị riêng - Vector riêng <a class="anchor" id="c11"></a>

#### Sử dụng thuật toán [Power iteration](https://en.wikipedia.org/wiki/Power_iteration) và Hotelling's Deflation [[1](https://web.stanford.edu/~lmackey/papers/deflation-nips08.pdf), [2](https://www.robots.ox.ac.uk/~sjrob/Teaching/EngComp/ecl4.pdf)]

In [ ]:
def eigen_power_iteration(A, n_iter=1000, eps=10e-5):
    # Create a random vector
    b_k = np.random.rand(A.shape[1])
    b_k_pre = np.zeros(A.shape[1])

    # Repeat the process of finding an eigenvector for a maximum of n_iter times
    for _ in range(int(n_iter)):
        # Calculate the dot product of matrix A and eigenvector
        numerator = np.dot(A, b_k)
        denominator = np.linalg.norm(numerator)
        
        b_k = numerator / denominator
        
        # Terminate the loop when the change in the vector is negligible
        if np.all(np.abs(b_k - b_k_pre) < eps):
            break
        
        b_k_pre = b_k
        
    # Find eigen value (Rayleigh quotient iteration)
    lamb = ((b_k @ A) @ b_k) / (b_k @ b_k)

    return lamb, b_k # lamb: eigen value, b_k: eigen vector


def my_eigens(A, n_iter=1000, eps=10e-5):
    eigenvalues = []
    eigenvectors = []
    
    n_rows = A.shape[0]
    
    # Find all possible eigenvalues and eigenvectors
    for _ in range(n_rows):
        lamb, b_k = eigen_power_iteration(A, n_iter, eps)
        
        eigenvalues.append(lamb)
        eigenvectors.append(b_k)
        
        # Hotelling's deflation
        b_k = b_k.reshape(-1, 1) # Convert to column vector
        A = A - lamb * (b_k @ b_k.T)
        
    return np.array(eigenvalues), np.array(eigenvectors).T

In [ ]:
my_eigenvalues, my_eigenvectors = my_eigens(A)

my_eigenvalues, my_eigenvectors

In [ ]:
# Evaluate the first pair of eigenvalue and eigenvector
my_lamb = my_eigenvalues[0]
my_X = my_eigenvectors[:, 0].reshape(-1, 1)

A @ my_X, my_lamb * my_X

In [ ]:
is_close(A @ my_X, my_lamb * my_X)

#### Thư viện `np.linalg`

In [ ]:
np_eigenvalues, np_eigenvectors = np.linalg.eig(A)

np_eigenvalues, np_eigenvectors

In [ ]:
# Evaluate the first pari of eigenvalue and eigenvector
np_lamb = np_eigenvalues[0]
np_X = np_eigenvectors[:, 0].reshape(-1, 1)

A @ np_X, np_lamb * np_X

In [ ]:
is_close(A @ np_X, np_lamb * np_X)

### Chéo hóa <a class="anchor" id="c12"></a>

In [ ]:
def my_diag(A):
    # Find eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    # Order eigenvectors by their corresponding eigenvalues
    sorted_idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_idx]
    eigenvectors = eigenvectors[:, sorted_idx]
    
    P_matrix = eigenvectors
    P_inv_matrix = np.linalg.inv(P_matrix)
    D_matrix = np.diag(eigenvalues)
    
    return P_matrix, D_matrix, P_inv_matrix

In [ ]:
np_eigenvalues, np_eigenvectors

In [ ]:
np_eigenvalues[np.argsort(np_eigenvalues)[::-1]], np_eigenvectors[:, np.argsort(np_eigenvalues)[::-1]]

In [ ]:
P, D, P_inv = my_diag(A)
(P, D, P_inv)

In [ ]:
D, P_inv @ A @ P

In [ ]:
is_close(D, P_inv @ A @ P)

### Chéo hóa trực giao <a class="anchor" id="c13"></a>

In [ ]:
def my_orth_diag(A):
    # Find eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    # Order eigenvectors by their corresponding eigenvalues
    sorted_idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_idx]
    eigenvectors = eigenvectors[:, sorted_idx]
    
    # Orthogonal
    Q_matrix, _ = np.linalg.qr(eigenvectors)
    
    Q_transpose = Q_matrix.T
    D_matrix = np.diag(eigenvalues)
    
    return Q_matrix, D_matrix, Q_transpose

In [ ]:
(Q, D, Q_T) = my_orth_diag(A)
(Q, D, Q_T)

In [ ]:
D, Q_T @ A @ Q

In [ ]:
is_close(D, Q_T @ A @ Q)

---

## Đồ án 2: Image Processing <a class="anchor" id="c2"></a>

### Nội dung đồ án <a class="anchor" id="c21"></a>

Nhắc lại: Trong đồ án 1, bạn đã được giới thiệu rằng ảnh được lưu trữ dưới dạng ma trận các điểm ảnh. Mỗi điểm ảnh có thể là một giá trị (ảnh xám) hoặc một vector (ảnh màu).

Trong đồ án này, bạn được yêu cầu thực hiện các chức năng xử lý ảnh cơ bản sau:
    
1. Thay đổi độ sáng cho ảnh (1 điểm)
- Tức là cộng thêm 1 lượng màu.
- Càng về 255, màu càng sáng lên
- Cộng thêm cho mỗi kênh màu 1 lượng là alpha -> Phép toán cộng ma trận với 1 scalar
- Đảm bảo sau khi cộng thì các giá trị <= 255

![img](https://imgur.com/oJ8bTv7.jpg)

2. Thay đổi độ tương phản (1 điểm)
- Tức là nhân ma trận với 1 scalar
- Vẫn phải đảm bảo sau khi nhân

![img](https://imgur.com/wl8MSu3.jpg)

3. Lật ảnh (ngang - dọc) (1 điểm)
- revert vector cột/dòng.

![img](https://imgur.com/MOOvIhN.jpg)

4. Chuyển đổi ảnh RGB thành ảnh xám/sepia (2 điểm)

- Ảnh xám
- Ảnh chỉ có 1 kênh màu -> Trung bình cộng 3 kênh màu thành 1 kênh màu. -> Thay đổi sự đóng góp của 3 kênh màu.

![img](https://imgur.com/XEfRXWE.jpg)

- Ảnh sepia

![img](https://imgur.com/YXUPjHY.jpg)

Keywords: convert RGB to grayscale/sepia

5. Làm mờ/sắc nét ảnh (2 điểm)

- Làm mờ (Thể hiện ở cả ảnh xám và ảnh màu)
  - Pixel đó = Trung bình cộng của 9 pixels xung quanh tính cả nó.

![img](https://imgur.com/wZT4vUa.jpg)

- Làm sắc nét

![img](https://imgur.com/H2Fq4Ne.jpg)

Tham khảo tại [đây](https://en.wikipedia.org/wiki/Kernel_(image_processing))

1. Cắt ảnh theo kích thước (cắt ở trung tâm) (1 điểm)
- Indexing

![img](https://imgur.com/fXebjfO.jpg)

7. Cắt ảnh theo khung hình tròn (1 điểm)

![img](https://imgur.com/DEpimhC.jpg)

8. Viết hàm main xử lý (1 điểm) với các yêu cầu sau:

- Cho phép người dùng nhập vào tên tập tin ảnh mỗi khi hàm main được thực thi.
- Cho phép người dùng lựa chọn chức năng xử lý ảnh (từ 1 đến 7, đối với chức năng 4 cho phép lựa chọn giữa lật ngang hoặc lật dọc). Lựa chọn 0 cho phép thực hiện tất cả chức năng với tên file đầu ra tương ứng với từng chức năng. Ví dụ:
    - Đầu vào: `cat.png`
    - Chức năng: Làm mờ
    - Đầu ra: `cat_blur.png`

Trong đồ án này, bạn <font style="color:red">**CHỈ ĐƯỢC PHÉP**</font> sử dụng các thư viện sau: `PIL`, `numpy`, `matplotlib`.

Cụ thể, nếu đề yêu cầu bạn viết ra chức năng đó, thì bạn phải thực sự viết ra chức năng đó chứ không phải gọi hàm có sẵn.

- Các bạn sử dụng `PIL` (`open(), save()` từ `Image`) để đọc và ghi; `Matplotlib` (`imshow()` từ `pyplot`) để hiển thị ảnh.

- Được phép sử dụng thư viện `NumPy` tùy ý.

Lưu ý: Để được **điểm tối đa** cho từng chức năng, thời gian thực thi của mỗi chức năng phải nằm trong khoảng thời gian chấp nhận được. Ví dụ với chức năng làm mờ (phức tạp nhất) có thời gian thực thi trên ảnh với kích thước $512 \times 512$ là dưới 15 giây.

#### Nâng cao - Không bắt buộc (Cộng 0.5 điểm vào điểm đồ án 2)

Sinh viên thực cắt nội dung ảnh theo khung được áp lên, với khung là 2 hình ellip chéo nhau như các ảnh sau:

- Khung là 2 hình ellip chéo nhau

![img](https://i.imgur.com/fPlYioC.png)


### Quy định bài nộp <a class="anchor" id="c22"></a>

* Thực hiện toàn bộ bài làm trên 1 tập tin Jupyter Notebook (.ipynb) hoặc Python (.py)


* Bạn nộp tập tin `MSSV.zip` được nén từ thư mục MSSV chứa các tập tin sau:
    1. Báo cáo toàn bộ bài làm: `MSSV.pdf`
    2. Mã nguồn: `MSSV.ipynb` hoặc `MSSV.py`


* Trong đó, nội dung tập tin báo cáo gồm có:
    - Thông tin cá nhân: Họ và tên, MSSV
    - Liệt kê các chức năng đã hoàn thành
    - Ý tưởng thực hiện, mô tả các hàm chức năng
    - Hình ảnh kết quả với từng chức năng
    - Báo cáo cần có số trang và tài liệu tham khảo

* Ví dụ minh họa cây thư mục bài nộp sau khi giải nén tập tin `MSSV.zip` như sau:
```
MSSV
├── MSSV.pdf
└── MSSV.ipynb hoặc MSSV.py
```

### Quy định chấm bài <a class="anchor" id="c23"></a>

Đây là đồ án chiếm 15%.

Những trường hợp sau đây sẽ bị 0 điểm toàn bộ đồ án:

- Sử dụng các thư viện và các hàm xử lý ảnh có sẵn mà không được cho phép
- Nộp sai quy định
- Không có báo cáo
- Thực thi mã nguồn báo lỗi

<font style="color:red">**LƯU Ý: SAO CHÉP BÀI LÀM CỦA NHAU SẼ BỊ 0 ĐIỂM TOÀN BỘ PHẦN THỰC HÀNH**</font>